In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-16 19:59:56,958 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet90.model')
model.init_sims(replace=True)

2017-06-16 19:59:57,557 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet90.model
2017-06-16 20:00:08,859 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet90.model.wv.* with mmap=None
2017-06-16 20:00:08,860 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet90.model.wv.syn0.npy with mmap=None
2017-06-16 20:00:18,336 : INFO : setting ignored attribute syn0norm to None
2017-06-16 20:00:18,374 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet90.model.syn1neg.npy with mmap=None
2017-06-16 20:00:59,386 : INFO : setting ignored attribute cum_table to None
2017-06-16 20:01:01,493 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet90.model
2017-06-16 20:03:21,100 : INFO : precomputing L2-norms of word weight vectors


In [3]:
num_features= 300

In [4]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [5]:
df.agreeabeness_m.value_counts()

1    542
0    497
Name: agreeabeness_m, dtype: int64

In [6]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.5, random_state=42)

In [8]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['agreeabeness_m']
    evaluate_prediction(predictions, target)

In [9]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [10]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [11]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [12]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-16 20:12:08,475 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:08,506 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:11,631 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:12,008 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:12,574 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:13,555 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:13,659 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:13,942 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:14,144 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:14,207 : WARNING : cannot compute similarity with no input []
2017-06-16 20:12:15,487 : WARNING : cannot compute similarity with no input []


CPU times: user 4.9 s, sys: 379 ms, total: 5.28 s
Wall time: 9.95 s


In [13]:
del model

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [15]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['agreeabeness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   19.6s finished


CPU times: user 4.03 s, sys: 346 ms, total: 4.37 s
Wall time: 25.9 s


In [16]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['agreeabeness_m']

In [17]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,126,114
1,117,163


In [18]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.52      0.53      0.52       240
          1       0.59      0.58      0.59       280

avg / total       0.56      0.56      0.56       520



In [19]:
accuracy_score(yt, yp)

0.55576923076923079

In [20]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [21]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [22]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       0.62      0.57      0.59       280
          1       0.54      0.59      0.56       239

avg / total       0.58      0.58      0.58       519



In [23]:
print accuracy_score(ytt, ytp)

0.578034682081
